In [1]:
import pandas as pd

In [7]:
from scale.dataset import read_csv

In [5]:
labels = pd.read_csv('../count_matrix_data/GSE96769_cell_names_matrix.csv', sep=',').set_index('cell_id')
# count, feature, barcode = read_mtx('mouse_atlas_cicero')
cell_types = labels['cell_types'].unique()
cell_barcodes = {}
for cell_type in cell_types:
    cell_barcodes[cell_type] = list(labels[labels['cell_types'] == cell_type].index)

In [26]:
count, feature, barcode = read_csv('../count_matrix_data/GSE96769_scATACseq_matrix.csv')

In [27]:
names_to_index = {
    code: index for index, code in enumerate(barcode)
}

In [34]:
barcode[481]

'cell_482'

In [35]:
labels.shape

(2953, 2)

In [37]:
count.shape

(2947, 489671)

In [38]:
def get_names(cell_names):
    return [
        names_to_index[code] for code in cell_names if code in names_to_index
    ]

In [46]:
import os

In [47]:
os.makedirs('hemato_data')

In [50]:
import numpy as np

for cell_type, code in cell_barcodes.items():
    cell_by_feature = np.asarray(count[get_names(code)].sum(axis=0)).flatten()
    feature_threshold = cell_by_feature[np.argsort(cell_by_feature)[-30000]]
    print(f'{cell_type}: {feature_threshold}')
    filtered_features = (cell_by_feature > 0) & (cell_by_feature >= feature_threshold)
    print(f'{cell_type}: filtered {np.sum(filtered_features)}')
    output = pd.DataFrame(feature[filtered_features])
    output.to_csv(
        os.path.join('hemato_data', f'imputed_data_raw_{cell_type.replace(" ", "_").replace("/", "_")}.txt'),
        header=None,
        index=None,
        sep='\t'
    )

HSC: 26.0
HSC: filtered 30629
MPP: 8.0
MPP: filtered 30583
LMPP: 11.0
LMPP: filtered 30688
CMP: 52.0
CMP: filtered 30479
CLP: 4.0
CLP: filtered 36723
MEP: 21.0
MEP: filtered 31309
GMP: 17.0
GMP: filtered 30957
pDC: 14.0
pDC: filtered 30935
Monocyte: 7.0
Monocyte: filtered 31068
UNK: 7.0
UNK: filtered 34559
GMP-3: 4.0
GMP-3: filtered 30978
GMP-2: 5.0
GMP-2: filtered 36634
GMP-1: 9.0
GMP-1: filtered 30784


In [44]:
output

,0
0,Peak_69
1,Peak_151
2,Peak_332
3,Peak_351
4,Peak_459
...,...
30779,Peak_179408
30780,Peak_101499
30781,Peak_205388
30782,Peak_145200
